In [1]:
from ctgan import CTGAN
from ct_pool_match import load_tabular_data, train
from ctgan import load_demo
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch
import pandas as pd
import numpy as np
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [2]:
import tensorflow as tf
print(torch.cuda.is_available())

True


In [3]:
real_data = pd.read_csv("cicids2018.csv")

In [4]:
scaler = StandardScaler()
label = LabelEncoder()
# data[data.columns[:-1]] = scaler.fit_transform(data[data.columns[:-1]])
real_data[real_data.columns[:-1]] = np.array(real_data[real_data.columns[:-1]], dtype=float)
real_data.replace([np.inf, -np.inf], np.nan, inplace=True)
#data.iloc[:, -1] = label.fit_transform(data.iloc[:, -1])
real_data.dropna(inplace=True)
real_data.head(3).T

,0,1,2
Dst Port,80.0,80.0,80.0
Protocol,6.0,6.0,6.0
Flow Duration,5007793.0,8938396.0,52700.0
Tot Fwd Pkts,4.0,2.0,5.0
Tot Bwd Pkts,4.0,0.0,3.0
...,...,...,...
Idle Mean,0.0,8938396.0,0.0
Idle Std,0.0,0.0,0.0
Idle Max,0.0,8938396.0,0.0
Idle Min,0.0,8938396.0,0.0


In [5]:

def get_discrete_columns(dataframe, max_unique_values=80):
    """
    Hàm để xác định các cột rời rạc (discrete), bao gồm các cột có số lượng giá trị unique ít hơn hoặc bằng max_unique_values.
    Ngoài ra, có thể chuyển đổi kiểu dữ liệu của các cột rời rạc sang string.
    
    - dataframe: DataFrame chứa dữ liệu
    - max_unique_values: Số lượng giá trị unique tối đa để coi là discrete (mặc định là 80)
    - convert_to_string: Nếu True, chuyển kiểu dữ liệu của các cột rời rạc sang string.
    
    Trả về:
    - discrete_columns: Danh sách tên các cột rời rạc.
    - discrete_indices: Danh sách chỉ số (index) của các cột rời rạc.
    """
    discrete_columns = []
    discrete_indices = []
    
    for col in dataframe.columns:
        unique_values = dataframe[col].nunique()
        print(f"Column '{col}' - Number of unique values: {unique_values}")
        # Chọn các cột có kiểu dữ liệu số và số lượng giá trị unique nhỏ hơn hoặc bằng max_unique_values
        if dataframe[col].dtype in ['float64', 'int64'] and unique_values <= max_unique_values:
            discrete_columns.append(col)
            discrete_indices.append(dataframe.columns.get_loc(col))
            
            dataframe[col] = dataframe[col].astype(str)
            print(f"Column '{col}' converted to string.")
    
    return discrete_columns, discrete_indices 


# Ví dụ với dataset của bạn
discrete_columns, discrete_indices = get_discrete_columns(real_data)
discrete_columns.append("Target")
print("Discrete Columns:", discrete_columns)
print("Indices of Discrete Columns:", discrete_indices)

Column 'Dst Port' - Number of unique values: 37447
Column 'Protocol' - Number of unique values: 3
Column 'Protocol' converted to string.
Column 'Flow Duration' - Number of unique values: 472240
Column 'Tot Fwd Pkts' - Number of unique values: 704
Column 'Tot Bwd Pkts' - Number of unique values: 942
Column 'TotLen Fwd Pkts' - Number of unique values: 6486
Column 'TotLen Bwd Pkts' - Number of unique values: 19272
Column 'Fwd Pkt Len Max' - Number of unique values: 1453
Column 'Fwd Pkt Len Min' - Number of unique values: 171
Column 'Fwd Pkt Len Mean' - Number of unique values: 21898
Column 'Fwd Pkt Len Std' - Number of unique values: 39819
Column 'Bwd Pkt Len Max' - Number of unique values: 1240
Column 'Bwd Pkt Len Min' - Number of unique values: 317
Column 'Bwd Pkt Len Mean' - Number of unique values: 33207
Column 'Bwd Pkt Len Std' - Number of unique values: 41211
Column 'Flow Byts/s' - Number of unique values: 512726
Column 'Flow Pkts/s' - Number of unique values: 493873
Column 'Flow IA

In [6]:
ctgan = CTGAN()

In [7]:
ctgan.load_model("2018\ctgan_model_epoch_10.pth")
#ctgan.fit(real_data, discrete_columns)

c:\Final Project in Uni\CTGAN\ctgan\synthesizers\ctgan.py:687: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


Model loaded from 2018\ctgan_model_epoch_10.pth


In [8]:
def get_unique_labels(dataframe, label_column):
    """
    Trả về mảng các giá trị nhãn duy nhất từ một cột trong tập dữ liệu.

    Args:
        dataframe (pd.DataFrame): Tập dữ liệu đầu vào.
        label_column (str): Tên cột chứa nhãn.

    Returns:
        list: Danh sách các nhãn duy nhất.
    """
    if label_column not in dataframe.columns:
        raise ValueError(f"Cột '{label_column}' không tồn tại trong tập dữ liệu.")
    
    unique_labels = dataframe[label_column].unique()
    return unique_labels.tolist()

In [9]:
unique_labels = get_unique_labels(real_data, 'Target')
unique_labels

['DoS attacks-GoldenEye',
 'Infilteration',
 'SSH-Bruteforce',
 'Bot',
 'DoS attacks-Hulk',
 'DDoS attacks-LOIC-HTTP',
 'DDOS attack-HOIC',
 'Benign']

In [10]:
def sample_by_label(ctgan, real_data, label_column):
    """
    Tạo dữ liệu tổng hợp bằng cách sinh một số lượng mẫu bằng với số lượng mẫu của từng nhãn trong dữ liệu thực.

    Args:
        ctgan (CTGAN): Đối tượng CTGAN đã được huấn luyện.
        real_data (pd.DataFrame): Tập dữ liệu thực.
        label_column (str): Tên cột chứa nhãn.

    Returns:
        pd.DataFrame: Dữ liệu tổng hợp với số lượng mẫu tương ứng cho từng nhãn.
    """
    # Kiểm tra nếu cột nhãn tồn tại
    if label_column not in real_data.columns:
        raise ValueError(f"Cột '{label_column}' không tồn tại trong tập dữ liệu.")
    
    # Lấy danh sách các nhãn duy nhất và số lượng mẫu của từng nhãn
    label_counts = real_data[label_column].value_counts()
    synthetic_data = []

    # Sinh dữ liệu cho từng nhãn
    for label, count in label_counts.items():
        print(f"Sinh {count} mẫu cho nhãn: {label}")
        batch_size = ctgan._batch_size  # Sử dụng batch_size từ CTGAN
        total_synthetic = 0

        current_synthetic = []
        while total_synthetic < count:
            remaining = min(batch_size, count - total_synthetic)
            synthetic_samples = ctgan.sample_by_label(
                n=remaining, 
                label_column=label_column, 
                label_value=label
            )
            synthetic_samples_df = pd.DataFrame(synthetic_samples)
            current_synthetic.append(synthetic_samples_df)
            total_synthetic += len(synthetic_samples_df)
        
        # Kết hợp các mẫu tổng hợp cho nhãn này
        current_synthetic = pd.concat(current_synthetic, ignore_index=True)
        
        # Kiểm tra và điều chỉnh số lượng mẫu
        if total_synthetic > count:
            # Nếu nhiều hơn, cắt bớt
            current_synthetic = current_synthetic.iloc[:count]
            print(f"Đã cắt bớt {total_synthetic - count} mẫu cho nhãn: {label}")
        elif total_synthetic < count:
            # Nếu ít hơn, sinh thêm mẫu
            print(f"Thiếu {count - total_synthetic} mẫu, tiếp tục sinh...")
            remaining = count - total_synthetic
            additional_samples = ctgan.sample_by_label(
                n=remaining, 
                label_column=label_column, 
                label_value=label
            )
            additional_samples_df = pd.DataFrame(additional_samples)
            current_synthetic = pd.concat([current_synthetic, additional_samples_df], ignore_index=True)
        
        # Thêm dữ liệu của nhãn này vào danh sách tổng hợp
        synthetic_data.append(current_synthetic)
        print(f"Hoàn thành {len(current_synthetic)} mẫu cho nhãn: {label}")
    
    # Kết hợp tất cả các mẫu tổng hợp
    synthetic_data = pd.concat(synthetic_data, ignore_index=True)
    return synthetic_data


In [11]:
def print_class_distribution(real_data, synthetic_data, label_column):
    """
    In ra số lượng mẫu của từng class (nhãn) trong cả tập gốc và tập sinh.

    Args:
        real_data (pd.DataFrame): Tập dữ liệu thực.
        synthetic_data (pd.DataFrame): Tập dữ liệu sinh.
        label_column (str): Tên cột chứa nhãn.
    """
    # Đếm số lượng mẫu của từng nhãn trong tập gốc
    real_counts = real_data[label_column].value_counts()

    # Đếm số lượng mẫu của từng nhãn trong tập sinh
    synthetic_counts = synthetic_data[label_column].value_counts()

    print("Số lượng mẫu của từng class:")
    print("-" * 40)
    print(f"{'Class':<15}{'Real Count':<15}{'Synthetic Count'}")
    print("-" * 40)

    # Lấy danh sách tất cả các nhãn (cả trong tập gốc và tập sinh)
    all_classes = set(real_counts.index).union(set(synthetic_counts.index))

    # In ra số lượng mẫu cho từng nhãn
    for cls in sorted(all_classes):
        real_count = real_counts.get(cls, 0)  # Lấy số lượng mẫu từ tập gốc
        synthetic_count = synthetic_counts.get(cls, 0)  # Lấy số lượng mẫu từ tập sinh
        print(f"{cls:<15}{real_count:<15}{synthetic_count}")


In [12]:
# Create synthetic data
##synthetic_data = sample_by_label(ctgan=ctgan, real_data=real_data, label_column='Target')

In [13]:
#print_class_distribution(real_data=real_data, synthetic_data=synthetic_data, label_column='Target')

In [14]:
#synthetic_data.to_csv("ctgan_80_same_samplepclass.csv", index=False)

In [15]:
def preprocess_data(data):
    """
    Tiền xử lý dữ liệu bằng cách chuẩn hóa các cột số và mã hóa các cột danh mục.
    
    Args:
        data (pd.DataFrame): Dữ liệu đầu vào.
    
    Returns:
        tuple: 
            - pd.DataFrame: Dữ liệu đã được tiền xử lý.
            - LabelEncoder: Bộ mã hóa nhãn được áp dụng.
            - dict: Ánh xạ giữa nhãn gốc và mã hóa.
    """
    data = data.copy()  # Tạo bản sao để tránh thay đổi dữ liệu gốc
    
    # Lấy danh sách các cột numeric và categorical
    numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns
    categorical_columns = data.select_dtypes(include=['object', 'category']).columns

    # Chuẩn hóa các cột số
    if len(numerical_columns) > 0:
        scaler = StandardScaler()
        data[numerical_columns] = scaler.fit_transform(data[numerical_columns])
    
    encoder = LabelEncoder()
    label_mapping = {}  # Lưu ánh xạ nhãn gốc -> mã hóa
    # Mã hóa các cột danh mục
    if len(categorical_columns) > 0:
        for col in categorical_columns:
            data[col] = encoder.fit_transform(data[col])
            # Tạo ánh xạ nhãn gốc -> mã hóa cho cột nhãn
            if col == 'Target':  # Giả định cột nhãn tên là 'Target'
                label_mapping = dict(zip(encoder.classes_, range(len(encoder.classes_))))
                label_mapping = {v: k for k, v in label_mapping.items()}
    return data, label_mapping

In [16]:
import os
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
class Args:
    input_dim = real_data.shape[1] - 1
    dim_noise = ctgan._embedding_dim
    batch_size = 256  # Kích thước batch
    epochs = 10  # Số epochs
    print_freq = 10  # Tần suất in kết quả
    # Tham số cho mô hình Generator
    generator_lr = 2e-4  # Learning rate cho generator
    match = 'grad' 
    match_coeff = 0.001  # Hệ số cho match_loss
    # Các tham số tối ưu hóa
    lr = 5e-6  # Learning rate chung cho mô hình
    momentum = 0.9  # Momentum
    weight_decay = 5e-4  # Weight decay cho optimizer
    num_classes = len(unique_labels)  # Số lớp trong dataset
    # Tham số cho dữ liệu và đường dẫn
    data_path = 'your_data.csv'  # Đường dẫn đến dữ liệu (cần thay đổi)
    output_dir = './models'  # Đường dẫn lưu trữ mô hình
    logs_dir = './logs/'  # Thư mục lưu logs
    data_dir = './data'  # Thư mục chứa dữ liệu
    # Các tham số phụ trợ khác
    match_aug = False  # Cờ để bật/tắt augmentation trong matching
    eval_lr = 0.01  # Learning rate cho evaluation (nếu có)
    epochs_match_train = 16
    fc = True
    bias = False
    metric = 'l1'  # Số epoch huấn luyện match model


if not os.path.exists(Args.output_dir):
    os.makedirs(Args.output_dir)

real_traindata, label_mapping = preprocess_data(real_data)
# Load dữ liệu

real_traindata


,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Target
0,-0.516442,2,-0.015060,-0.009234,-0.019505,-0.005371,-0.018549,0.577287,-0.429321,0.764633,...,4,-0.074557,-0.062011,-0.085762,-0.058870,-0.016633,-0.001754,-0.006701,-0.308360,4
1,-0.516442,2,-0.008237,-0.011212,-0.043111,-0.012764,-0.022538,-0.741551,-0.429321,-0.907988,...,4,-0.074557,-0.062011,-0.085762,-0.058870,0.008012,-0.001754,0.001980,0.194343,4
2,-0.516442,2,-0.023662,-0.008244,-0.025406,-0.005032,-0.018549,0.637667,-0.429321,0.491371,...,4,-0.074557,-0.062011,-0.085762,-0.058870,-0.016633,-0.001754,-0.006701,-0.308360,4
3,-0.516442,2,-0.020586,-0.011212,-0.043111,-0.012764,-0.022538,-0.741551,-0.429321,-0.907988,...,4,-0.074557,-0.062011,-0.085762,-0.058870,-0.016633,-0.001754,-0.006701,-0.308360,4
4,-0.516442,2,-0.022204,-0.011212,-0.043111,-0.012764,-0.022538,-0.741551,-0.429321,-0.907988,...,4,-0.074557,-0.062011,-0.085762,-0.058870,-0.016633,-0.001754,-0.006701,-0.308360,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854397,-0.520470,0,0.171779,0.049141,-0.043111,-0.012764,-0.022538,-0.741551,-0.429321,-0.907988,...,0,14.615509,30.337597,20.835478,2.018185,0.026975,0.015559,0.011728,0.403439,0
854398,2.227774,2,-0.016546,-0.008244,-0.025406,0.003892,-0.021282,2.229807,-0.429321,2.106760,...,4,-0.074557,-0.062011,-0.085762,-0.058870,-0.016633,-0.001754,-0.006701,-0.308360,0
854399,2.061460,2,-0.023662,-0.011212,-0.037210,-0.012087,-0.022538,-0.620790,-0.429321,-0.601677,...,1,-0.074557,-0.062011,-0.085762,-0.058870,-0.016633,-0.001754,-0.006701,-0.308360,0
854400,-0.498164,2,-0.023752,-0.010223,-0.043111,-0.011392,-0.022538,-0.595367,-0.429321,-0.494199,...,1,-0.074557,-0.062011,-0.085762,-0.058870,-0.016633,-0.001754,-0.006701,-0.308360,0


In [17]:
print(Args.num_classes)


8


In [18]:
print(Args.input_dim)

78


In [19]:
mean = torch.zeros(ctgan._batch_size, ctgan._embedding_dim).to(ctgan._device)
std = mean + 1
noise = torch.normal(mean=mean, std=std) 

In [20]:
noise

tensor([[ 0.7672, -0.0217, -1.1356,  ..., -0.5825, -0.6961, -1.2064],
        [ 0.8499,  1.2450,  0.9346,  ...,  1.3762,  0.7817,  1.0228],
        [ 1.2143,  0.9375, -0.2057,  ...,  0.1729, -0.5489, -0.5395],
        ...,
        [-0.2866,  0.7696,  1.2007,  ...,  0.4298, -1.5690, -0.5389],
        [-2.6488,  0.9089, -1.0404,  ..., -0.7328,  0.5827,  0.9714],
        [ 0.3050, -0.9296,  0.2524,  ...,  0.9067, -0.7279,  1.1022]],
       device='cuda:0')

In [21]:
label_mapping

{0: 'Benign',
 1: 'Bot',
 2: 'DDOS attack-HOIC',
 3: 'DDoS attacks-LOIC-HTTP',
 4: 'DoS attacks-GoldenEye',
 5: 'DoS attacks-Hulk',
 6: 'Infilteration',
 7: 'SSH-Bruteforce'}

In [22]:
# Kiểm tra cột target và mã hóa
print(real_traindata[real_traindata.columns[-1]].head())  # Giá trị ban đầu
print(real_traindata['Target'].head())      # Giá trị sau mã hóa


0    4
1    4
2    4
3    4
4    4
Name: Target, dtype: int32
0    4
1    4
2    4
3    4
4    4
Name: Target, dtype: int32


In [23]:
train_data = TensorDataset(torch.tensor(real_traindata.values, dtype=torch.float32))
trainloader = DataLoader(train_data, batch_size=Args.batch_size, shuffle=True, drop_last=True)

In [24]:
img_real = real_traindata.iloc[:, :-1].values 

In [25]:
discriminator = ctgan.getDiscriminator(img_real,1)

input_dim shape:  78


In [26]:
import torch.nn as nn
from torch import optim
# Khởi tạo optimizer
generator = ctgan._generator
optim_g = ctgan.optim_G
optim_d = ctgan.optim_D
if(optim_g == None):
    optim_g = optim.Adam(
            generator.parameters(),
            lr=ctgan._generator_lr,
            betas=(0.5, 0.9),
            weight_decay=ctgan._generator_decay,
        )
optim_d = ctgan.optim_D
if(optim_d == None):
    optim_d = optim.Adam(
            discriminator.parameters(),
            lr=ctgan._discriminator_lr,
            betas=(0.5, 0.9),
            weight_decay=ctgan._discriminator_decay,
    )
criterion = nn.CrossEntropyLoss() # type: ignore
model_folder = "2018"
model_save_path = "ct_dim_new_2018_20e.pth"
# Huấn luyện discriminator với dữ liệu từ CTGAN
train(Args, ctgan,model_folder, model_save_path,generator, discriminator,optim_g, optim_d, trainloader, label_mapping, criterion)

Random model selected: mlp_deep
[Epoch 0 Iter 10] G Loss: 0.756 (2.097) | D Loss: 8.148 (10.148) | Logits Loss: 0.824 (0.602) | D Acc: 1.172 (13.281)
[Epoch 0 Iter 20] G Loss: 2.878 (2.294) | D Loss: 8.076 (9.162) | Logits Loss: 0.686 (0.664) | D Acc: 0.781 (10.762)
[Epoch 0 Iter 30] G Loss: 2.243 (2.439) | D Loss: 6.286 (8.213) | Logits Loss: 1.510 (0.718) | D Acc: 3.125 (9.141)
[Epoch 0 Iter 40] G Loss: 2.041 (2.474) | D Loss: 5.879 (7.332) | Logits Loss: 1.090 (0.775) | D Acc: 1.562 (9.795)
[Epoch 0 Iter 50] G Loss: 1.098 (2.534) | D Loss: 3.087 (6.834) | Logits Loss: 0.820 (0.833) | D Acc: 52.344 (9.156)
[Epoch 0 Iter 60] G Loss: 2.238 (2.537) | D Loss: 5.529 (6.533) | Logits Loss: 1.052 (0.870) | D Acc: 25.000 (11.458)
[Epoch 0 Iter 70] G Loss: 1.916 (2.575) | D Loss: 3.056 (6.209) | Logits Loss: 0.604 (0.907) | D Acc: 1.562 (11.596)
[Epoch 0 Iter 80] G Loss: 3.173 (2.622) | D Loss: 5.138 (6.023) | Logits Loss: 0.969 (0.928) | D Acc: 0.391 (10.288)
[Epoch 0 Iter 90] G Loss: 2.130 

KeyboardInterrupt: 

In [ ]:
synthetic_data = ctgan.sample(int(len(real_data)*0.8))

In [ ]:
synthetic_data.to_csv("2018/ctgandim_10e_2018_80pt.csv", index=False)